# 10주 1강: Pandas Basics 2

# Combining Datasets: Concat and Append

데이터를 다룰 때 가장 많이 하는 작업은 다른 소스의 데이터를 결합하는 것. DataFrame은 기본적으로 SQL같은 테이블 구조를 가지고 있기 때문에, SQL처럼 데이터를 병합하는 작업을 할 수 있고, 매우 효울적으로 작동함

가장 먼저 배울 것은 Concat과 Append입니다. 
1. Concat
- 동질성있는 데이터프레임을 합치는 concatenation
- ``Series`` 와 ``DataFrame``은 ``pd.concat`` 을 통해서 결합 가능
2. Append


In [1]:
import pandas as pd
import numpy as np

In [2]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


Jupyter에서는 하나의 dataframe을 보여주는 것이 기본이기 때문에, 여러개의 ``DataFrame``을 나란히 표시 하는 클래스도 정의. Jupyter의 ``_repr_html_`` methods를 사용.

In [3]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

## Recall: Concatenation of NumPy Arrays

``Series`` 와 ``DataFrame``의 Concatenation은 Numpy Array의 Concatenation인 np.concatenate와 매우 유사함. 

In [4]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

첫 번째 인수는 연결할 배열의 목록 또는 튜플. 또한 axis 키워드를 통해 연결될 축을 정의.

In [5]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## Simple Concatenation with ``pd.concat``

Pandas도 유사하게 ``pd.concat()``이라는 함수가 있음. 작동 방법은 ``np.concatenate``와 비슷하지만, 훨씬 다양한 기능을 제공.

```python
pd.concat(objs, axis=0, join='outer', ignore_index=False, keys=None, 
          levels=None, names=None, verify_integrity=False, sort=False, copy=True)
```

``pd.concat()`` 은 ``Series`` 나 ``DataFrame``의 단순 연결을 위해 사용. 

In [6]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

``DataFrame``도 연결 가능

In [7]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


기본적으로 이러한 연결은 옵션을 지정하지 않으면 행 단위로 이루어짐 (아래쪽에 연결됩니다). ``np.concatenate``와 마찬가지로``pd.concat``도 axis를 통해 다른 축으로 연결 가능함.

In [8]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis=1)") # or axis="column"

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis=1)
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

### Duplicate indices

``np.concatenate``와 ``pd.concat``의 가장 중요한 차이는 결과에 중복 인덱스가 있더라도 Pandas의 경우 인덱스가 유지된다는 것.

In [9]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


반복되는 인덱스가 있음. ``DataFrame`` 내에서 작동하는 방법이지만, 때때로 한 index에 한 개의 값(혹은 행/열)만 가지고 있기를 원하는 경우도 있음. 이를 처리하는 ``pd.concat()`` 의 method들이 존재함.

#### Catching the repeats as an error

단순히``pd.concat()`` 결과의 인덱스가 겹치지 않는지 확인하려면 ``verify_integrity`` 플래그를 지정 가능함.
True로 설정하면 중복 인덱스가있는 경우 연결에서 예외가 발생. 

In [10]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


#### Ignoring the index

Index 값 자체가 중요하지 않으면 단순히 무시하는 것이 좋음. 이 옵션은 ``ignore_index`` 플래그를 사용하여 지정 가능함.
이 값을 true로 설정하면 concatenate는 ``Series`` 새로운 정수 인덱스를 생성.

In [11]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


#### Adding MultiIndex keys

또 다른 옵션은``keys``옵션을 사용하여 데이터 소스에 대한 레이블을 지정하는 것. 결과는 데이터를 포함하는 MultiIndex를 가짐

In [12]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

### Concatenation with joins

방금 살펴본 간단한 예제에서는 주로``DataFrame``을 같은 column label만 가진 경우에만 생각. 그런데 실제로 서로 다른 소스의 데이터는 서로 다른 column name 집합을 가질 수 있음. ``pd.concat`` 은 이 경우에도 여러 옵션을 제공. 일부 열만 공통으로 갖는 다음 두``DataFrame``의 연결.

In [13]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

기본적으로 데이터를 사용할 수없는 항목은 NA 값으로 채워짐.
이를 변경하기 위해 concatenate 함수의``join`` 및 ``join_axes`` 매개 변수에 대한 여러 옵션 중 하나를 지정 가능함.
기본적으로 조인은 outer가 기본. 하지만 inner로 변경하면, 겹치는 열만 사용해서 concatenate를 해 줌.

In [14]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

### The ``append()`` method

직접 배열 연결이 매우 일반적이기 때문에``Series`` 및 ``DataFrame`` 객체에는 더 적은 키 입력으로 동일한 작업을 수행 할 수있는 ``append`` 함수가 정의되어 있음. 예를 들어 ``pd.concat([df1, df2])``를 호출하는 대신 단순히``df1.append(df2)``를 호출 할 수 있음.

In [15]:
display('df1', 'df2', 'df1.append(df2)')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Python list의``append()`` 및 ``extend()`` method와 달리 Pandas의 ``append()`` method는 원래 객체를 수정하지 않고 대신 새 객체를 생성. 또한 새로운 인덱스 와 데이터 버퍼를 생성하기 때문에 그리 효율적인 방법이 아님.
따라서 합친 이후 추가적인 처리를 하려는 경우는 ``DataFrame`` list를 통해 concat 함수를 사용하는 것이 좋음. 

# Combining Datasets: Merge and Join

Pandas의 강력함 중 하나는 아주 빠른 고성능 인 메모리 merge와 및 join을 구현. 

편의를 위해 아까 정의한 ``display ()``기능을 재정의하는 것으로 시작. 

In [16]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Relational Algebra

``pd.merge()``는 관계형 데이터를 조작하기위한 공식적인 규칙인 Relational Algebra의 하위 집합. 또한 대부분의 데이터베이스에서 사용할 수있는 기능과 같은 개념. 관계형 대수 접근 방식의 강점은 모든 데이터 세트에서 더 복잡한 연산의 구성 요소가되는 몇 가지 기본 연산이 정의되어 있음. DB나 Pandas에서는 이러한 기본 연산을 통해 복잡한 연산을 단순화시킬 수 있음.

Pandas는 SQL의 병합 기능과 비슷한 기능을 pd.merge와 pd.join 두 가지로 정의해 둠 

## Categories of Joins

``pd.merge()`` 함수는 1:1, 1:n, n:n join과 같은 여러가지 유형의 join을 구현해 둠. 세 가지 연산방식은 모두 pd.merge 인터페이스를 사용해서 수행할 수 있고, 입력하는 데이터의 형태에 따라 달라짐. 

### One-to-one joins

아마도 가장 간단한 병합 표현 유형은 일대일 조인. 이는 여러면에서 아까 다룬 column에 대한 concat과 비슷. 
아래 두 개의 DataFrame은 employee column을 공통으로 가짐.

In [19]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


``pd.merge()``는 이를 employee를 기준으로 병합.

In [20]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


``pd.merge ()``함수는 각``DataFrame``에 employee열이 있음을 인식하고이 열을 키로 사용하여 자동으로 조인함.

그리고 그 결과로 두 df가 결합된 새 DataFrame을 리턴해줌.
이 때 각 열의 항목 순서가 반드시 유지되는 것은 아님. 예를 들어 employee열의 순서는 df1, df2, df_merge 모두 다를 수 있음 
또한 아주 특수한 경우를 제외하고 merge는 인덱스를 삭제해서 정수 인덱스를 새로 부여.

### Many-to-one joins

1:n 두 키 열 중 하나에 중복 항목이 포함 된 경우. ``DataFrame``은 해당 중복 항목을 적절하게 남겨 둠.

In [22]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')
# df3(다) : df4(1)

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

리턴되는 dataframe에는 부서에 따른 "supervisor" 정보다 모두 들어가 있음. Engineering은 두 명이 있으므로, 두 개에 같은 값 (Guido)가 채워졌다는 것을 확인해.

### Many-to-many joins

n:n 병합은 개념적으로 약간 어려울 수 있습니다만, pandas에서는 잘 정의되어 있음.
왼쪽 및 오른쪽 배열의 키 열에 모두 중복 항목이 포함 된 경우 n:n 병합을 수행.

In [23]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


## Specification of the Merge Key

pd.merge는 두 입력 사이에 하나 이상의 일치하는 열 이름을 찾고 이것을 키로 사용.
그러나 종종 열 이름이 너무 일치하지 않지만 같은 데이터인 경우가 있음. ``pd.merge ()``는 이를 처리하기위한 다양한 옵션을 제공.

### The ``on`` keyword

가장 간단하게는 열 이름 또는 열 이름 list를 on 키워드에 입력하여 병합 기준 열을 명시 적으로 지정할 수 있음.

In [24]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

이 옵션은 왼쪽 및 오른쪽 ``DataFrame``모두에 지정된 열과 같은 이름이 있는 경우에만 작동.

### The ``left_on`` and ``right_on`` keywords

때로는 열 이름이 다른 두 개의 데이터 세트를 병합 할 수 있음. left_on 과 right_on 키워드를 사용하여 각 dataframe의 열 이름을 지정할 수 있음

In [25]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

결과에는 원하는 경우 삭제할 수있는 중복 열이 있음. 
- drop을 통해 삭제 필요

In [25]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### The ``left_index`` and ``right_index`` keywords

때로는 열을 병합하는 대신 인덱스를 병합하고 싶을 때가 있음. 사실 인덱스를 병합하는 것이 더 빠름. 

In [26]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


``pd.merge()``에 left_index와 right_index 플래그를 지정하여 인덱스를 병합 키로 사용 가능함.

In [27]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


``DataFrame``은 또 이러한 index 기반 병합을 수행하는 ``join()`` method도 존재

In [28]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


인덱스와 열을 혼합해서 병합키로 쓰려면``left_index``를``right_on``과 결합하거나 ``left_on`` 을 ``right_index``와 결합.

In [29]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

이러한 모든 옵션은 여러 인덱스 또는 여러 열에서도 작동.

## Specifying Set Arithmetic for Joins

이런 병합은 집합의 연산 방법도 지정할 수 있음.

In [30]:
# 하나의 키 열에는 값이 있고, 다른 하나에는 없는 경우
# 공통된 "name"이 column이 Mary밖에 없음
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

inner join : 기본적으로 결과에는 두 입력 집합의 교집합이 사용.
- 이러한 병합 방법은 how 키워드로 지정 가능함
<br>
how키워드
- outer : 합집합을 사용하고, 모든 누락값을 NA로 채움
- left : 왼쪽 항목을 기준으로 병합
- right : 오른쪽 항목을 기준으로 병합

In [33]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


In [34]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

In [35]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')", "pd.merge(df6, df7, how='right')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

pd.merge(df6, df7, how='right')
     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer

## Overlapping Column Names: The ``suffixes`` Keyword

마지막으로 두 입력 DataFrame에 충돌하는 열 이름이 있을 경우.

In [36]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

두 개의 충돌하는 열 이름이 있기 때문에 merge 함수는 자동으로 접미사 ``_x`` 와 ``_y``를 추가하여 이름을 겹치지 않게 해 줌. 이러한 기본값이 적절하지 않은 경우 ``suffixes`` 키워드를 사용하여 사용자 지정 접미사를 사용 가능.

In [37]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

# Aggregation and Grouping

대용량 데이터를 분석하기 위해서는 평균, 총합, 중간값, 최대값, 최소값 등의 요약통계를 보는 것이 매우 중요함. Pandas는 groupby라는 기능을 통해 정해진 속성별로 이런 대표값을 구할 수 있음 

In [38]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Planets Data

In [39]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [41]:
planets.head()
# 2014년까지 발견 된 1,000 개 이상의 외행성에 대한 세부 정보

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


## Simple Aggregation in Pandas

NumPy 배열과 마찬가지로 Pandas Series는 단일 값을 반환함

In [43]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [44]:
ser.sum()

2.811925491708157

In [45]:
ser.mean()

0.5623850983416314

``DataFrame``의 경우 기본적으로 aggregation을 사용하면 각 열마다 결과를 반환.

In [46]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [47]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

``axis`` argument를 지정하면 row마다의 평균을 구할 수도 있음.

In [48]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Pandas Series 와 DataFrame은 기본적인 aggregation method 들을 모두 구현해 둠. 그리고 하나식 계산할 필요 없이 각 열에 대해 몇 가지 공통 집계를 계산하고 결과를 반환하는 편리한 describe() method가 있음. 

In [49]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


이는 데이터 세트의 전체 속성을 이해하는 데 유용한 방법
- year열을 보면, 외행성은 1989년 부터 발견되었지만 알려진 별의 절반은 2010 년 이후까지 발견되지 않음.
- 2009년 3월 7일에 역사적인 사건 : 케플러 우주망원경을 발사. 즉, 2009년부터 발견된 수많은 행성들은 케플러 우주망원경의 공이 큼.

아래와같은 Aggreegation들이 정의되어 있음 

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

또한 DataFrame과 Series에 모두 사용할 수 있음

## GroupBy: Split, Apply, Combine

데이터 전체의 aggregation 또한 유용하게 쓰입니다만, 때로는 조건부로 구한 값들이 더 필요한 경우가 있음
- 2000년대 이전에 구한 별들의 평균 거리와 그 이후의 평균 거리의 차이. 
<br>이런 경우는 groupby라는 함수를 통해 할 수 있습니다.


### Split, apply, combine

- split 단계는 지정된 키의 값에 따라 DataFrame 을 분리하고 그룹화
- apply 단계는 개별 그룹 내에서 일반적으로 aggregation, transformation, filtering 같은 것을 수행
- combine 단계는 이러한 작업의 결과를 출력 배열로 병합

사실 앞서 다룬 마스킹, 집계 및 병합 명령의 일부 조합을 사용하여 수동으로이 작업이 가능. 하지만 groupby는 이러한 중간 분할 단계를 직접 정의할 필요도 없고, 같은 문법으로 다양한 분할을 자동으로 수행할 수 있음! 즉, 각 단계를 추상화하여 사용자는 안에서 어떻게 작동하는지 전혀 생각 할 필요가 없음

In [50]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


가장 기본적인 split-apply-combine 연산은``DataFrame``의``groupby ()``메서드를 사용하여 원하는 키 열을 지정하면 됨.

In [51]:
df.groupby('key')

리턴되는 결과물이 DataFrame 혹은 여러 DataFrame의 집합이 아니라 ``DataFrameGroupBy`` 객체.
이 객체는 실제로 apply 단계를 사용하기 전에 계산을 수행하기 위한 DataFrame 조회용 객체

In [52]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


### The GroupBy object

GroupBy 객체는 고도로 추상화 되어있고, 굉장히 유연하게 사용 가능함. 때때로는 DataFrame의 집합처럼 간단히 쓸 수도 있고, 내부적으로 복잡한 함수를 적용할 수 있음.

#### Column indexing

GroupBy object는 DataFrame과 동일한 방식으로 column indexing을 지원

In [53]:
planets.groupby('method')

In [54]:
planets.groupby('method')['orbital_period']

In [53]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

#### Iteration over groups

``GroupBy`` object는 직접적으로 group에 대한 iteration도 지원함.

In [55]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


이 기능은 특정 작업을 수동으로 수행하는 데 유용 할 수 있지만 apply() moethod를 사용하는 것이 일반적으로 훨씬 더 빠름.

#### Dispatch methods
``GroupBy`` obejct에 명시적으로 구현되지 않은 모든 methods는 그룹별로 Series나 DataFrame에 구현된 연산으로 작동
-  예를 들어서 describe()는 GroupBy에 구현되지 않았지만, 아래와 같이 Groupby object의요약통계를 만드는 데 쓸 수 있음

In [56]:
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.0
       Eclipse Timing Variations           9.0
       Imaging                            38.0
       Microlensing                       23.0
       Orbital Brightness Modulation       3.0
                                         ...  
max    Pulsar Timing                    2011.0
       Pulsation Timing Variations      2007.0
       Radial Velocity                  2014.0
       Transit                          2014.0
       Transit Timing Variations        2014.0
Length: 80, dtype: float64

각 개별 그룹에 describe가 적용되고 결과가 ``GroupBy``내에서 병합되어 리턴 

### Aggregate, filter, transform, apply

위에 다룬 단순한 사용 이외에도 ``GroupBy`` object는 aggregate(), filter(), transform() 및 apply() method가 정의되어 있어서, 다양한 기능을 효율적으로 사용할 수 있음.

In [57]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### Aggregation

aggregate() method는 동시에 여러가지 연산을 수행할 수 있음. 이 aggregate의 인자는 string, function, 그리고 위의 것들의 list 등을 모두 사용할 수 있음

In [58]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

또 다른 유용한 패턴은 사전을 통해서 해당 열에 적용 할 작업을 지정하는 것

In [59]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


In [60]:
df.groupby('key').aggregate({'data1': ['min', "mean"],
                             'data2': ["median", 'max']})

data1       data2    
      min mean median max
key                      
A       0  1.5    4.0   5
B       1  2.5    3.5   7
C       2  3.5    6.0   9

#### Filtering
필터링 작업을 사용하면 그룹 속성에 따라 데이터를 삭제 가능. 
- 표준 편차가 일부 임계 값보다 큰 모든 그룹을 유지할 수 있음

In [61]:
def filter_func(x):
    return x['data2'].std() > 4

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

필터 함수는 그룹이 필터링을 통과하는지 여부를 확인해 Boolean 값을 반환

#### Transformation

Transformation은 원 데이터를 결합 후 변환한 결과를 원 DataFrame과 같은 형태로 리턴. 
- 데이터값을 그 평균값으로 빼주는 아래와 같은 작업을 생각할 수 있음

In [62]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


#### The apply() method

``apply()`` method를 사용하면 GroupBy 결과에 임의의 함수를 적용 할 수 있음. 
함수는``DataFrame``을 가져와 Pandas 객체 (DataFrame 이나 Series) 또는 스칼라 변수를 반환해야 함. 
결합 작업은 반환되는 출력 유형에 맞게 자동으로 조정됨.

- data2값의 합을 통해서 data1을 정규화하는 norm_by_data2 함수를 정의해서 apply 해 봄

In [63]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


``GroupBy``의 ``apply()``는 매우 자유로움. 
- DataFrame을 가져와 Pandas 객체 (DataFrame 이나 Series) 또는 스칼라 변수를 반환하는 조건만 맞으면 어떤 함수도 가능. 
- 정의한 모든 함수를 다 쓸 수 있음

### Specifying the split key

훨씬 복잡하고 다양한 방법으로 Groupby를 수행 가능함

#### A list, array, series, or index providing the grouping keys

Groupby의 key는 그룹을 지정하는 임의의 list로 주어질 수 있음

In [64]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(L).sum()
   data1  data2
0      7     17
1      4      3
2      4      7

In [65]:
display('df', "df.groupby(df['key']).sum()")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(df['key']).sum()
     data1  data2
key              
A        3      8
B        5      7
C        7     12

#### A dictionary or series mapping index to group

또 다른 방법은 인덱스 값을 그룹 키에 매핑하는 사전을 사용.
- B와 C를 동일 그룹으로 묶고 싶을 때

In [66]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
key,,
consonant,12,19


#### Any Python function

매핑과 마찬가지로 인덱스 값을 입력하고 그룹 이름을 출력하는 모든 Python 함수를 사용할 수 있음.

In [67]:
display('df2', 'df2.groupby(str.lower).mean()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
key,,
a,1.5,4.0


#### A list of valid keys

또한 여러 개의 항목을 결합하여 다중 인덱스로 그룹화 가능

In [68]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
key,key,,
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


### Grouping example

- 예를 들어, 몇 줄의 Python 코드로 위의 데이터를 결합하고 발견 된 행성과 발견 방법을 10 년 단위로 계산 가능

In [69]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
